In [ ]:
import torch
import whisper

model = whisper.load_model("tiny.en").cpu().eval()

decoder = model.decoder

kv_cache, hooks = decoder.install_kv_cache_hooks()

tokenizer = whisper.tokenizer.get_tokenizer(True)

audio = whisper.load_audio("sample.mp3")
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio).unsqueeze(0)

audio_features = model.encoder(mel)

tokens = torch.tensor([[tokenizer.sot, tokenizer.no_timestamps]])

logits = decoder(tokens, audio_features, kv_cache=kv_cache)

result = tokens[0].tolist()

for _ in range(200):

    next_token = logits[:, -1].argmax(-1)

    if next_token.item() == tokenizer.eot:
        break

    result.append(next_token.item())

    logits = decoder(next_token.unsqueeze(0), audio_features, kv_cache=kv_cache)

print(tokenizer.decode(result))
